In [1]:
print("OK")

OK


In [112]:
print("Hello world \n My world is ...")

Hello world 
 My world is ...


In [2]:
import os 
from dotenv import load_dotenv 
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter 

In [7]:
%cd ..

e:\#GenAI_Krish_Nail\7-End-to-End GenAI projects using LLOPS and Deployment\Interview-Question-Creator


In [65]:
# load_dotenv()
# OpenAI_api_key = os.environ.get('OPENAI_API_KEY') 
# OpenAI_api_key

In [5]:
os.environ['OPENAI_API_KEY'] = OpenAI_api_key 

In [114]:
os.getcwd() 

'e:\\#GenAI_Krish_Nail\\7-End-to-End GenAI projects using LLOPS and Deployment\\Interview-Question-Creator'

In [135]:
%cd Interview-Question-Creator/

e:\#GenAI_Krish_Nail\7-End-to-End GenAI projects using LLOPS and Deployment\Interview-Question-Creator


In [4]:
%pwd

'e:\\#GenAI_Krish_Nail\\7-End-to-End GenAI projects using LLOPS and Deployment\\Interview-Question-Creator\\research'

In [8]:
file_path = "data/stats.pdf" 
loader = PyPDFLoader(file_path) 
data = loader.load() 

In [9]:
data

[Document(metadata={'source': 'data/stats.pdf', 'page': 0}, page_content='Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na\nperfect\nnegative\nlinear\nrelationship.\nCorrelation\nis\nimportant\nin\nstatistics\nbecause\nit\ncan\nbe\nused\nto\n1.\nIdentify\nrelationships\nbetween\nvariables:\nCorrelation\ncan\nbe\nused\nto\nidentify\nwhether\nthere\nis\na\nrelationship\nbetween\ntwo\nvariables,\nand\nif\nso,\nwhether\nthe\nrelationship\nis\npositive\nor\nnegative.\nThis

In [118]:
len(data)

3

In [10]:
question_gen = "" 
for page in data: 
    question_gen += page.page_content 

In [11]:
chunking = TokenTextSplitter(

    model_name = 'gpt-3.5-turbo', chunk_size = 10000,
    chunk_overlap = 200 
)

In [12]:
after_chunking = chunking.split_text(question_gen) 

In [13]:
len(after_chunking)  

1

In [37]:
type(after_chunking)

list

# Document Term matrix 

In [14]:
from langchain.docstore.document import Document 

In [15]:
doc_question_after_chunking = [Document (page_content=
                                          x )for x in after_chunking ] 

In [42]:
# print(doc_question_after_chunking)  
print(type(doc_question_after_chunking[0]))  

<class 'langchain_core.documents.base.Document'>


In [16]:
chunking_answer = TokenTextSplitter(

    model_name = 'gpt-3.5-turbo', chunk_size = 1000,
    chunk_overlap = 100 
)

In [17]:
document_answer_gen = chunking_answer.split_documents(doc_question_after_chunking) 

In [85]:
len(document_answer_gen) 

2

# Initializing the LLM model and 

In [30]:
from langchain.chat_models import ChatOpenAI 
from langchain_openai import ChatOpenAI 

In [43]:
llm_ques_gen_pipeline = ChatOpenAI(model = 'gpt-3.5-turbo') 

In [45]:
prompt_template = """

You are an expert at creating questions based on coding materials and documentations. 
Your goal is to prepare a coder or programmer for their exam and coding tests. 
You do this by asking questions about the ext below: 

-------
{text}
-------
Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS: 


"""




In [46]:
from langchain.prompts import PromptTemplate 




In [47]:
PROMPT_QUESTIONS = PromptTemplate(template = prompt_template, input_variable =['text']) 

In [48]:
refine_template = (

    """ 
You are an exper tat creating prcic questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test. 
We have  received some proactice questions to a certain extent: {existing_answer}

------
{text}
------


Given the new context, refine the original questions in English. 
If the context is not helpful, please provide the original questions. 

QUESTIONS: 

    """


)

In [49]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
input_variable =['existing_answer', 'text' ] ,
template = refine_template 
)

# Creating a chain 

In [50]:
from langchain.chains import load_summarize_chain 

In [51]:
ques_gen_chain = load_summarize_chain (
    llm = llm_ques_gen_pipeline, 
    chain_type = "refine", verbose = False, 
    question_prompt = PROMPT_QUESTIONS, refine_prompt = REFINE_PROMPT_QUESTIONS
    
)

# defining the wrapper 

In [54]:
ques = ques_gen_chain.run(document_answer_gen) 

print(ques)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [62]:
from langchain.embeddings.openai import OpenAIEmbeddings 
# from langchain-openai import OpenAIEmbeddings  

embedding_model = OpenAIEmbeddings() 
from langchain.vectorstores import FAISS 
vectro_store = FAISS.from_documents (document_answer_gen, embedding_model) 

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [55]:
# Initializing LLM for answer generation 
llm_answer_gen = ChatOpenAI(temperature= 0.1, model = "gpt-3.5-turbo") 

In [63]:

ques_list = ques.split ("\n")


NameError: name 'ques' is not defined

In [56]:
from langchain.chains import RetrievalQA 

In [58]:
answer_generation_chain = RetrievalQA.from_chain_type (llm= llm_answer_gen,  chain_type = "stuff", 
                                                       retriever = vectro_store.as_retriever()) 

NameError: name 'vectro_store' is not defined

In [ ]:
# wrapper 

for question in ques_list:
    print("Question", question)
    answer = answer_generation_chain.run(question) 
    print("Answer: ", answer)
    print("------------------------------------------------------------------\\n\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\n\n")
        f.write("Answer: " + answer + "\\n")
        f.write("-------------------------------------------------------------\\n\n") 

        